In [1]:
import pandas as pd
import numpy as np
import string
import unidecode
import re
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline

from gensim.models import KeyedVectors
from gensim.models import Word2Vec

from nltk.util import ngrams

from pprint import pprint

import spacy
nlp = spacy.load('pt_core_news_lg') #pt_core_news_sm

SEED = 46

/home/ricardo/anaconda3/envs/pln_env/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
df = pd.read_csv('dados/SICK_BR_Train.csv', delimiter=';', encoding='mac_roman')

df.head(5)

,pair_ID,sentence_A,sentence_B,entailment_label,relatedness_score,entailment_AB,entailment_BA,sentence_A_original,sentence_B_original,sentence_A_dataset,sentence_B_dataset
0,1,Um grupo de crianças está brincando em um quin...,Um grupo de meninos em um quintal está brincan...,NEUTRAL,4.5,A_neutral_B,B_neutral_A,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,FLICKR,FLICKR
1,2,Um grupo de crianças está brincando na casa e ...,Um grupo de crianças está brincando em um quin...,NEUTRAL,3.2,A_contradicts_B,B_neutral_A,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,FLICKR,FLICKR
2,3,Os meninos jovens estão brincando ao ar livre ...,As crianças estão brincando ao ar livre perto ...,ENTAILMENT,4.7,A_entails_B,B_entails_A,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,FLICKR,FLICKR
3,4,Os meninos jovens estão brincando ao ar livre ...,Não tem nenhum menino brincando ao ar livre e ...,CONTRADICTION,3.6,A_contradicts_B,B_contradicts_A,The young boys are playing outdoors and the ma...,There is no boy playing outdoors and there is ...,FLICKR,FLICKR
4,5,As crianças estão brincando ao ar livre perto ...,Um grupo de crianças está brincando em um quin...,NEUTRAL,3.4,A_neutral_B,B_neutral_A,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,FLICKR,FLICKR


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8840 entries, 0 to 8839
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pair_ID              8840 non-null   int64  
 1   sentence_A           8840 non-null   object 
 2   sentence_B           8840 non-null   object 
 3   entailment_label     8840 non-null   object 
 4   relatedness_score    8840 non-null   float64
 5   entailment_AB        8840 non-null   object 
 6   entailment_BA        8840 non-null   object 
 7   sentence_A_original  8840 non-null   object 
 8   sentence_B_original  8840 non-null   object 
 9   sentence_A_dataset   8840 non-null   object 
 10  sentence_B_dataset   8840 non-null   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 759.8+ KB


In [4]:
scale_label_mapper = {'NEUTRAL':0, 
                      'ENTAILMENT':1,
                      'CONTRADICTION':2}

df['label'] = df['entailment_label'].replace(scale_label_mapper)

df.head(5)

,pair_ID,sentence_A,sentence_B,entailment_label,relatedness_score,entailment_AB,entailment_BA,sentence_A_original,sentence_B_original,sentence_A_dataset,sentence_B_dataset,label
0,1,Um grupo de crianças está brincando em um quin...,Um grupo de meninos em um quintal está brincan...,NEUTRAL,4.5,A_neutral_B,B_neutral_A,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,FLICKR,FLICKR,0
1,2,Um grupo de crianças está brincando na casa e ...,Um grupo de crianças está brincando em um quin...,NEUTRAL,3.2,A_contradicts_B,B_neutral_A,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,FLICKR,FLICKR,0
2,3,Os meninos jovens estão brincando ao ar livre ...,As crianças estão brincando ao ar livre perto ...,ENTAILMENT,4.7,A_entails_B,B_entails_A,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,FLICKR,FLICKR,1
3,4,Os meninos jovens estão brincando ao ar livre ...,Não tem nenhum menino brincando ao ar livre e ...,CONTRADICTION,3.6,A_contradicts_B,B_contradicts_A,The young boys are playing outdoors and the ma...,There is no boy playing outdoors and there is ...,FLICKR,FLICKR,2
4,5,As crianças estão brincando ao ar livre perto ...,Um grupo de crianças está brincando em um quin...,NEUTRAL,3.4,A_neutral_B,B_neutral_A,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,FLICKR,FLICKR,0


### Balanceamento das classes (downsample)

In [5]:
# Verificando o balanceamento das classes
df['label'].value_counts()

0    5261
1    2488
2    1091
Name: label, dtype: int64

In [6]:
from sklearn.utils import resample

df_maior = df[df['label']==0]
qtd_maior = len(df_maior)

df_meio = df[df['label']==1]
qtd_meio = len(df_meio)

df_menor = df[df['label']==2]
qtd_menor = len(df_menor)

df_maior_down = resample(df_maior, replace=False, n_samples=qtd_menor, random_state=SEED)
df_meio_down = resample(df_meio, replace=False, n_samples=qtd_menor, random_state=SEED)

df = pd.concat([df_menor, df_meio_down, df_maior_down])

df['label'].value_counts()

0    1091
2    1091
1    1091
Name: label, dtype: int64

In [7]:
df = df.drop(columns=['relatedness_score', 'entailment_label',
                      'entailment_AB', 'entailment_BA',
                      'sentence_A_original', 'sentence_B_original', 
                      'sentence_A_dataset', 'sentence_B_dataset'])

df.head()

,pair_ID,sentence_A,sentence_B,label
3,4,Os meninos jovens estão brincando ao ar livre ...,Não tem nenhum menino brincando ao ar livre e ...,2
12,13,Dois cachorros estão lutando e se abraçando,Não tem nenhum cachorro lutando e abraçando,2
19,20,Não há nenhum homem de jaqueta preta fazendo t...,Uma pessoa de blusa preta está fazendo truques...,2
22,23,Ninguém está dirigindo uma bicicleta com uma roda,Uma pessoa está andando de bicicleta em uma só...,2
30,31,Um homem com uma camisa de time está enterrand...,Não tem nenhum homem jogando a bola em um jogo...,2


### Pré-processamento, Steemer, Lematização e POS Tagger, NER

In [8]:
def normalizar_texto(sentenca):
    sentenca = re.sub(r'\n', '', sentenca) #remove as quebras de linha
    sentenca = re.sub(r'\t', ' ', sentenca) #substitui tabulações por um espaço em branco
    sentenca = re.sub(r'\s+', ' ', sentenca, flags=re.I) #substitui um ou mais espaços em branco por um espaço
    sentenca = re.sub('["‘’“”…]', '', sentenca) #remove aspas e apóstofres
    return unidecode.unidecode(sentenca.lower()) #remove acentos


In [9]:
def tokenizar_texto(sentenca):
    tokens = nlp(sentenca)
    ners = [(entity.text, entity.label_) for entity in tokens.ents]
    chunkers = [(token.text, token.dep_, token.head.text, token.head.pos_, [child for child in token.children]) for token in tokens]
    tokens = [token for token in tokens if not token.is_punct]
    tokens = [token for token in tokens if not token.is_stop] 
    postags = [token.pos_ for token in tokens]
    lemmas = [token.lemma_.lower().strip() for token in tokens]
    
    ners = ' '.join([str(item) for item in ners]) 
    postags = ' '.join([str(item) for item in postags]) 
    lemmas = ' '.join([str(item) for item in lemmas]) 
    chunkers = ' '.join([str(item) for item in chunkers]) 
    
    return ners, postags, lemmas, chunkers

In [10]:
def word_grams(words, n):
    grams = []
    for ngram in ngrams(words, n):
        grams.append(' '.join(str(i) for i in ngram))
    grams = ' '.join([str(item) for item in grams]) 
    return grams

In [11]:
texto_exemplo = {}

for i in range(df.shape[0]):
    a_normalizado = normalizar_texto(df.iloc[i, 1])
    b_normalizado = normalizar_texto(df.iloc[i, 2])
                                     
    a_ners, a_postags, a_lemmas, a_chunkers = tokenizar_texto(a_normalizado)
    b_ners, b_postags, b_lemmas, b_chunkers = tokenizar_texto(b_normalizado)
                                     
    sentence_a_unigrama = word_grams(a_normalizado.split(), 1)
    sentence_b_unigrama = word_grams(b_normalizado.split(), 1)
    
    sentence_a_bigrama = word_grams(a_normalizado.split(), 2)
    sentence_b_bigrama = word_grams(b_normalizado.split(), 2)
    
    texto_exemplo[i] = {'id': df.iloc[i, 0],
                        'sentenca_a': df.iloc[i, 1],
                        'sentenca_b': df.iloc[i, 2],
                        'label': df.iloc[i, 3],
                        'sentenca_a_normalizado': a_normalizado,
                        'sentenca_b_normalizado': b_normalizado,
                        'sentenca_a_ner': a_ners,
                        'sentenca_b_ner': b_ners,
                        'sentenca_a_postag': a_postags,
                        'sentenca_b_postag': b_postags,
                        'sentenca_a_lemma': a_lemmas,
                        'sentenca_b_lemma': b_lemmas,
                        'sentenca_a_chunker': a_chunkers,
                        'sentenca_b_chunker': b_chunkers,
                        'sentenca_a_unigrama': sentence_a_unigrama,
                        'sentenca_b_unigrama': sentence_b_unigrama,
                        'sentenca_a_bigrama': sentence_a_bigrama,
                        'sentenca_b_bigrama': sentence_b_bigrama}

print(texto_exemplo[0]['sentenca_a_lemma'])

menino jovem estao brincar ar livrar homem sorrir


In [12]:
#sentimento

### Vetorização das features


In [13]:
EMBEDDING_FILE = 'dados/skip_s300.txt'
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=False, unicode_errors="ignore")

def word2vec_embedding(sentenca):
    word2vec_words = []
    for word in sentenca.split():
        try:
            word2vec_word = word2vec[word]
            word2vec_words.append(word2vec_word)
        except KeyError:
            'word ' + word + ' not in vocabulary'
    sentence_word2vec = list(map(sum, zip(*word2vec_words)))
    return sentence_word2vec

In [14]:
df_trab = pd.DataFrame(texto_exemplo).T

df_trab.tail()

,id,sentenca_a,sentenca_b,label,sentenca_a_normalizado,sentenca_b_normalizado,sentenca_a_ner,sentenca_b_ner,sentenca_a_postag,sentenca_b_postag,sentenca_a_lemma,sentenca_b_lemma,sentenca_a_chunker,sentenca_b_chunker,sentenca_a_unigrama,sentenca_b_unigrama,sentenca_a_bigrama,sentenca_b_bigrama
3268,9241,O homem de azul está posando para uma foto ao ...,A mulher de azul está posando para uma foto pe...,0,o homem de azul esta posando para uma foto ao ...,a mulher de azul esta posando para uma foto pe...,,,NOUN ADJ VERB NOUN NOUN,NOUN NOUN VERB NOUN NOUN,homem azul posar foto carro,mulher azul posar foto carro,"('o', 'det', 'homem', 'NOUN', []) ('homem', 'n...","('a', 'det', 'mulher', 'NOUN', []) ('mulher', ...",o homem de azul esta posando para uma foto ao ...,a mulher de azul esta posando para uma foto pe...,o homem homem de de azul azul esta esta posand...,a mulher mulher de de azul azul esta esta posa...
3269,3459,O homem está cortando um tronco de árvore com ...,Um homem está picando um tronco com um machado,0,o homem esta cortando um tronco de arvore com ...,um homem esta picando um tronco com um machado,,,NOUN VERB NOUN NOUN NOUN,NOUN VERB NOUN NOUN,homem cortar troncar arvorar machadar,homem picar troncar machadar,"('o', 'det', 'homem', 'NOUN', []) ('homem', 'R...","('um', 'det', 'homem', 'NOUN', []) ('homem', '...",o homem esta cortando um tronco de arvore com ...,um homem esta picando um tronco com um machado,o homem homem esta esta cortando cortando um u...,um homem homem esta esta picando picando um um...
3270,6730,Uma criança está esmagando uma bola de neve,Uma criança de laranja está brincando com uma ...,0,uma crianca esta esmagando uma bola de neve,uma crianca de laranja esta brincando com uma ...,,,NOUN VERB NOUN NOUN,NOUN NOUN VERB NOUN NOUN,crianca esmagar bolar nevar,crianca laranjo brincar bolar nevar,"('uma', 'det', 'crianca', 'NOUN', []) ('crianc...","('uma', 'det', 'crianca', 'NOUN', []) ('crianc...",uma crianca esta esmagando uma bola de neve,uma crianca de laranja esta brincando com uma ...,uma crianca crianca esta esta esmagando esmaga...,uma crianca crianca de de laranja laranja esta...
3271,6697,Um grupo de pessoas está sentado em ambos os l...,Um grupo de pessoas está parado em ambos os la...,0,um grupo de pessoas esta sentado em ambos os l...,um grupo de pessoas esta parado em ambos os la...,,,NOUN VERB NOUN NOUN NOUN ADJ,NOUN VERB NOUN NOUN NOUN ADJ,pessoa sentar lado estruturar pedrar vermelho,pessoa parar lado estruturar pedrar vermelho,"('um', 'det', 'grupo', 'NOUN', []) ('grupo', '...","('um', 'det', 'grupo', 'NOUN', []) ('grupo', '...",um grupo de pessoas esta sentado em ambos os l...,um grupo de pessoas esta parado em ambos os la...,um grupo grupo de de pessoas pessoas esta esta...,um grupo grupo de de pessoas pessoas esta esta...
3272,6549,Um menino pequenino está parado na floresta,Um menino pequenino está sentado na floresta,0,um menino pequenino esta parado na floresta,um menino pequenino esta sentado na floresta,,,NOUN ADJ VERB NOUN,NOUN ADJ VERB NOUN,menino pequenino parar florestar,menino pequenino sentar florestar,"('um', 'det', 'menino', 'NOUN', []) ('menino',...","('um', 'det', 'menino', 'NOUN', []) ('menino',...",um menino pequenino esta parado na floresta,um menino pequenino esta sentado na floresta,um menino menino pequenino pequenino esta esta...,um menino menino pequenino pequenino esta esta...


In [15]:
w2v_sentence_a = []
w2v_sentence_b = []

for i in range(df_trab.shape[0]):
    w2v_sentence_a.append(word2vec_embedding(df_trab.iloc[i, 4]))
    w2v_sentence_b.append(word2vec_embedding(df_trab.iloc[i, 5]))

df_w2v_sentenca_a = pd.DataFrame(w2v_sentence_a, columns=['sent_a_w2v_' + str(i) for i in range(1, len(w2v_sentence_a[1]) + 1)])
df_w2v_sentenca_b = pd.DataFrame(w2v_sentence_a, columns=['sent_b_w2v_' + str(i) for i in range(1, len(w2v_sentence_b[1]) + 1)])

print(df_w2v_sentenca_a.shape)
# df_w2v_sentenca_a.head()

(3273, 300)


In [16]:
tfidf = TfidfVectorizer(max_features=2000)

lemma_a_vector = tfidf.fit_transform(df_trab['sentenca_a_lemma'])
lemma_b_vector = tfidf.fit_transform(df_trab['sentenca_b_lemma'])
postag_a_vector = tfidf.fit_transform(df_trab['sentenca_a_postag'])
postag_b_vector = tfidf.fit_transform(df_trab['sentenca_b_postag'])
chunker_a_vector = tfidf.fit_transform(df_trab['sentenca_a_chunker'])
chunker_b_vector = tfidf.fit_transform(df_trab['sentenca_b_chunker'])
unigrama_a_vector = tfidf.fit_transform(df_trab['sentenca_a_unigrama'])
unigrama_b_vector = tfidf.fit_transform(df_trab['sentenca_b_unigrama'])
bigrama_a_vector = tfidf.fit_transform(df_trab['sentenca_a_bigrama'])
bigrama_b_vector = tfidf.fit_transform(df_trab['sentenca_b_bigrama'])

X = np.hstack((lemma_a_vector.todense(),
               lemma_b_vector.todense(), 
               postag_a_vector.todense(), 
               postag_b_vector.todense(),
               chunker_a_vector.todense(), 
               chunker_b_vector.todense(), 
               unigrama_a_vector.todense(), 
               unigrama_b_vector.todense(), 
               bigrama_a_vector.todense(),
               bigrama_b_vector.todense(),
               df_w2v_sentenca_a, 
               df_w2v_sentenca_b))

print(X.shape)

(3273, 15147)


### Treinamento do modelo e análise dos resultados

In [17]:
y = df_trab['label']
y = y.astype('int')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED, shuffle=True)

In [18]:
rfc_model = RandomForestClassifier(max_depth=15, min_samples_leaf=2, min_samples_split=7, random_state=SEED).fit(X_train, y_train)

print('Parameters currently in use:\n')
pprint(rfc_model.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 15,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 2,
 'min_samples_split': 7,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 46,
 'verbose': 0,
 'warm_start': False}


In [19]:
print('Random Forest (Treino): ',rfc_model.score(X_train, y_train))
y_pred = rfc_model.predict(X_test)
print('Random Forest (Teste): ', accuracy_score(y_test, y_pred))

Random Forest (Treino):  0.9711916193801833
Random Forest (Teste):  0.5763747454175153


In [20]:
cm = pd.DataFrame(confusion_matrix(y_test, y_pred), columns=scale_label_mapper.keys(), index=scale_label_mapper.keys())

print('Matriz de Confusão do Random Forest: \n', cm)
print('\n\nAvaliação do Random Forest: \n', classification_report(y_test, y_pred))

Matriz de Confusão do Random Forest: 
                NEUTRAL  ENTAILMENT  CONTRADICTION
NEUTRAL            111         176             48
ENTAILMENT         106         192             13
CONTRADICTION       36          37            263


Avaliação do Random Forest: 
               precision    recall  f1-score   support

           0       0.44      0.33      0.38       335
           1       0.47      0.62      0.54       311
           2       0.81      0.78      0.80       336

    accuracy                           0.58       982
   macro avg       0.57      0.58      0.57       982
weighted avg       0.58      0.58      0.57       982



#### Treinando e avaliando o Random Forest com validação cruzada

In [21]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

rfc_model = RandomForestClassifier(max_depth=15, min_samples_leaf=2, min_samples_split=7, random_state=SEED)

scores = cross_val_score(rfc_model, X, y, cv=cv)

score_medio = scores.mean()
score_desvio = scores.std()

print('Acurácia média (CV): %.3f' % score_medio)
print('Intervalo: [%.3f, %.3f]' % (score_medio - 2 * score_desvio, score_medio + 2 * score_desvio))

Acurácia média (CV): 0.576
Intervalo: [0.552, 0.599]


In [22]:
y_preds = cross_val_predict(rfc_model, X, y, cv=cv)

cm = pd.DataFrame(confusion_matrix(y, y_preds), columns=scale_label_mapper.keys(), index=scale_label_mapper.keys())

print('Matriz de Confusão do Random Forest (CV): \n', cm)
print('\n\nAvaliação do Random Forest (CV): \n', classification_report(y, y_preds))

Matriz de Confusão do Random Forest (CV): 
                NEUTRAL  ENTAILMENT  CONTRADICTION
NEUTRAL            406         499            186
ENTAILMENT         405         639             47
CONTRADICTION      128         124            839


Avaliação do Random Forest (CV): 
               precision    recall  f1-score   support

           0       0.43      0.37      0.40      1091
           1       0.51      0.59      0.54      1091
           2       0.78      0.77      0.78      1091

    accuracy                           0.58      3273
   macro avg       0.57      0.58      0.57      3273
weighted avg       0.57      0.58      0.57      3273



In [23]:
print('Parameters currently in use:\n')
pprint(rfc_model.get_params())


Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 15,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 2,
 'min_samples_split': 7,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 46,
 'verbose': 0,
 'warm_start': False}


### Ajustando os hiperparâmetros do RandomForest

In [24]:
# https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
# https://towardsdatascience.com/optimizing-hyperparameters-in-random-forest-classification-ec7741f9d3f6

hyperparams = {'n_estimators' : [50, 100, 150],
               'max_depth' : [10, 15],
               'min_samples_split' : [5, 7, 9],
               'min_samples_leaf' : [1, 2] 
              }

gs_model = GridSearchCV(RandomForestClassifier(random_state=SEED), hyperparams, cv = cv)

gs_model.fit(X, y)

print('Melhores parametros : ', gs_model.best_params_)

print('Melhor score: ', gs_model.best_score_)

results = pd.DataFrame(gs_model.cv_results_)

results.head(30)

Melhores parametros :  {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 7, 'n_estimators': 150}
Melhor score:  0.5915003384924248


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.594552,0.015685,0.029260,0.000151,10,1,5,50,"{'max_depth': 10, 'min_samples_leaf': 1, 'min_...",0.596947,0.586260,0.554198,0.579511,0.567278,0.576839,0.014861,25
1,3.047000,0.013573,0.038398,0.000188,10,1,5,100,"{'max_depth': 10, 'min_samples_leaf': 1, 'min_...",0.598473,0.592366,0.560305,0.590214,0.585627,0.585397,0.013209,13
2,4.482934,0.008807,0.047124,0.000148,10,1,5,150,"{'max_depth': 10, 'min_samples_leaf': 1, 'min_...",0.590840,0.616794,0.558779,0.585627,0.602446,0.590897,0.019305,2
3,1.575366,0.004644,0.029168,0.000114,10,1,7,50,"{'max_depth': 10, 'min_samples_leaf': 1, 'min_...",0.595420,0.607634,0.563359,0.588685,0.570336,0.585087,0.016234,14
4,3.011917,0.011227,0.038349,0.000227,10,1,7,100,"{'max_depth': 10, 'min_samples_leaf': 1, 'min_...",0.596947,0.609160,0.560305,0.582569,0.585627,0.586922,0.016273,9
5,4.442428,0.014459,0.046950,0.000079,10,1,7,150,"{'max_depth': 10, 'min_samples_leaf': 1, 'min_...",0.604580,0.619847,0.569466,0.581040,0.582569,0.591500,0.018162,1
6,1.559649,0.007770,0.029175,0.000060,10,1,9,50,"{'max_depth': 10, 'min_samples_leaf': 1, 'min_...",0.566412,0.603053,0.566412,0.562691,0.584098,0.576533,0.015212,27
7,2.992497,0.008299,0.038766,0.001543,10,1,9,100,"{'max_depth': 10, 'min_samples_leaf': 1, 'min_...",0.566412,0.604580,0.570992,0.564220,0.579511,0.577143,0.014686,24
8,4.409920,0.011050,0.046893,0.000251,10,1,9,150,"{'max_depth': 10, 'min_samples_leaf': 1, 'min_...",0.569466,0.610687,0.586260,0.581040,0.584098,0.586310,0.013496,10
9,1.569669,0.003811,0.030138,0.001883,10,2,5,50,"{'max_depth': 10, 'min_samples_leaf': 2, 'min_...",0.572519,0.584733,0.549618,0.576453,0.576453,0.571955,0.011857,33


In [25]:
hyperparams = {'n_estimators' : [50, 100, 150, 200], # number of trees in the foreset
               'max_features' : ['auto', 'sqrt'], # Method considered for splitting a node
               'max_depth' : [8, 10, 15, 20], # Maximum number of levels in each decision tree
               'min_samples_split' : [2, 5, 8, 12], # Minimum number of data points placed in a node before the node is split
               'min_samples_leaf' : [1, 2, 3, 4, 5], # Minimum number of data points allowed in a leaf node
               'bootstrap' : [True, False] # Method for sampling data points (with or without replacement)
              }

rs_model = RandomizedSearchCV(RandomForestClassifier(random_state=SEED), hyperparams, cv = cv, n_iter = 100)

rs_model.fit(X, y)

print('Melhores parametros : ', rs_model.best_params_)
print('Melhor score: ', gs_model.best_score_)
rs_model.best_estimator_

Melhores parametros :  {'n_estimators': 200, 'min_samples_split': 12, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 8, 'bootstrap': False}
Melhor score:  0.5915003384924248


RandomForestClassifier(bootstrap=False, max_depth=8, max_features='sqrt',
                       min_samples_leaf=4, min_samples_split=12,
                       n_estimators=200, random_state=46)

In [26]:
# def evaluate(model, test_features, test_labels):
#     predictions = model.predict(test_features)
#     errors = abs(predictions - test_labels)
#     mape = 100 * np.mean(errors / test_labels)
#     accuracy = 100 - mape
#     print('Model Performance')
#     print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
#     print('Accuracy = {:0.2f}%.'.format(accuracy))
#     return accuracy

# base_model = rfr_model
# base_model.fit(train_features, train_labels)
# base_accuracy = evaluate(base_model, test_features, test_labels)